<a href="https://colab.research.google.com/github/nipunidevinuwara/sequential_models/blob/main/IT21487484_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = "/content/drive/MyDrive/MLOM/data.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [ ]:
print(raw_text[0:1900])

project gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.org


title: alice's adventures in wonderland

author: lewis carroll

posting date: june 25, 2008 [ebook #11]
release date: march, 1994
[last updated: december 20, 2011]

language: english


*** start of this project gutenberg ebook alice's adventures in wonderland ***










alice's adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0




chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought alice 'without picture

In [ ]:
#create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
print(chars)
char_to_int = dict((c,i) for i,c in enumerate(chars))

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
len(chars)

58

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters : ", n_chars)
print("Total Vocab(Unique characters) : ", n_vocab)

Total characters :  163780
Total Vocab(Unique characters) :  58


In [ ]:
seq_length = 15 #this can be changed
#if we use 'set=10' then the next 15 characters will be selected after skipping 10 characters
dataX = []
dataY = [] #what we are trying to predict
for i in range(0, n_chars-seq_length,1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns= len(dataY)
print("Total patterns : ", n_patterns)

Total patterns :  163765


In [ ]:
import numpy

In [ ]:
X = numpy.reshape(dataX, (n_patterns, seq_length,1))
#rescale values from 0 to 1
X = X /float(n_vocab)
print(X)

[[[0.81034483]
  [0.84482759]
  [0.79310345]
  ...
  [0.77586207]
  [0.56896552]
  [0.62068966]]

 [[0.84482759]
  [0.79310345]
  [0.70689655]
  ...
  [0.56896552]
  [0.62068966]
  [0.84482759]]

 [[0.79310345]
  [0.70689655]
  [0.62068966]
  ...
  [0.62068966]
  [0.84482759]
  [0.65517241]]

 ...

 [[0.55172414]
  [0.56896552]
  [0.79310345]
  ...
  [0.79310345]
  [0.79310345]
  [0.72413793]]

 [[0.56896552]
  [0.79310345]
  [0.89655172]
  ...
  [0.79310345]
  [0.72413793]
  [0.86206897]]

 [[0.79310345]
  [0.89655172]
  [0.87931034]
  ...
  [0.72413793]
  [0.86206897]
  [0.22413793]]]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

ImportError: ignored

In [ ]:
y = keras.utils.to_categorical(dataY)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
#from keras.layers.attention.multi_head_attention import activation
from keras.api._v2.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2)) #to get rid of overfitting, randomly 20% neurons will be reduced
model.add(Dense(y.shape[1],activation='softmax'))#usually used in text generation purposes, will provide probabilities for each characters
model.compile(loss='categorical_crossentropy', optimizer='adam')#categorical crossentropy since we have multiple classes

#saving weights while training
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True,mode='min')
callbacks_list=[checkpoint]



In [ ]:
epochs = 10
batch_size = 128

model.fit(X, y , epochs=epochs, batch_size=batch_size,callbacks=callbacks_list)

Epoch 1/10
1280/1280 [==============================] - ETA: 0s - loss: 2.9862
Epoch 1: loss improved from inf to 2.98625, saving model to weights-improvement-01-2.9862.hdf5
1280/1280 [==============================] - 153s 117ms/step - loss: 2.9862
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1280/1280 [==============================] - ETA: 0s - loss: 2.7968
Epoch 2: loss improved from 2.98625 to 2.79682, saving model to weights-improvement-02-2.7968.hdf5
1280/1280 [==============================] - 151s 118ms/step - loss: 2.7968
Epoch 3/10
1280/1280 [==============================] - ETA: 0s - loss: 2.7185
Epoch 3: loss improved from 2.79682 to 2.71847, saving model to weights-improvement-03-2.7185.hdf5
1280/1280 [==============================] - 148s 116ms/step - loss: 2.7185
Epoch 4/10
1280/1280 [==============================] - ETA: 0s - loss: 2.6657
Epoch 4: loss improved from 2.71847 to 2.66570, saving model to weights-improvement-04-2.6657.hdf5
1280/1280 [==============================] - 149s 116ms/step - loss: 2.6657
Epoch 5/10
1280/1280 [==============================] - ETA: 0s - loss: 2.6197
Epoch 5: loss improved from 2.66570 to 2.61969, saving model to weights-improvement-05-2.6197.hdf5
1280/1280 [==============================] - 148s 116ms/step - loss: 2.

In [ ]:
filename ="weights-improvement-10-2.3958.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
int_to_char = {i: char for i, char in enumerate(chars)}

In [ ]:
print(len(dataX))
start = numpy.random.randint(0, len(dataX)-1)
print(start)
pattern=dataX[start]
print("Seed : ")
print("\"",''.join([int_to_char[value] for value in pattern]), "\"")

163765
69096
Seed : 
" on't much care  "


In [ ]:
length = 10
final=[]
for i in range(length):
  x=numpy.reshape(pattern,(1,len(pattern),1))
  #print(x)
  x = x / float(n_vocab)
  #print(x)
  prediction=model.predict(x,verbose=0)
  index=numpy.argmax(prediction)

  result=int_to_char[index]
  #print(result)

  final.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print(final)

['t', 'o', ' ', 't', 'h', 'e', ' ', 'c', 'a', 'r']
